In [3]:
import logging
import numpy as np
import os

import torch
import torch.nn as nn

from model import FCNet
from dataset import PolyDataset
from dataset import HTOCM

torch.manual_seed(42)
np.random.seed(42)

from tqdm import tqdm
import torch.optim as optim

# taken from https://github.com/hjmshi/PyTorch-LBFGS 
# from LBFGS import LBFGS, FullBatchLBFGS

# original github repo
# https://github.com/amirgholami/adahessian
# 
# lessw2020 update
# https://github.com/lessw2020/Best-Deep-Learning-Optimizers/tree/master/adahessian
from adahessian import Adahessian